# IBM DATA SCIENCE PROFESSIONAL CERTIFICATE CAPSTONE PROJECT
### This is Navie Huynh's capstone project for IBM's Data science professional certificate course

**Importing Pandas and Numpy**

In [23]:
import pandas as pd
import numpy as np

**Download BeautifulSoup4**

In [3]:
!conda install beautifulsoup4 --yes


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         156 KB
    beautifulsoup4-4.8.1       |           py36_0         153 KB
    soupsieve-1.9.5            |           py36_0          61 KB
    openssl-1.1.1d             |       h7b6447c_3         3.7 MB
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.5-py36_0                 

The following packages will b

**Download lxml and html5lib**

In [19]:
!conda install lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [20]:
!conda install html5lib

Solving environment: | 
  - conda-forge::html5lib-0.9999999-py36_0
  - defaults::html5lib-0.9999999-py36done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



**Import webscraping libraries**

In [17]:
import requests
from bs4 import BeautifulSoup

# Webscraping with BeautifulSoup and Pandas

### We will webscrape Canadian Postal/Borough/Neighborhood data from wikipedia:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


### First we will use the requests package to save the link into a python object

In [60]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

### Next we will use BeautifulSoup to extract/query the html object to find the desired wikipedia table

In [59]:
soup = BeautifulSoup(res.content,'html.parser')
table = soup.find_all('table')[0]

### Finally we will use pandas' 'read_html' method to create a pandas dataframe with the table created from BeautifulSoup. 

#### We will drop all rows where Borough is Not Assigned.  


In [79]:
df = pd.read_html(str(table),flavor='html5lib',header=0)[0]
df = df[df.Borough != 'Not assigned']
df

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


##### Note that the initial assignment of df_sorted does not remove the intital indexing from pd.read_html and so we call reset_index to reset the index of the new dataframe

In [80]:
df.reset_index(drop=True, inplace=True)
df

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


### Find all occurances where Neighborhood is not assigned and assign Borough value 

In [81]:
df[df['Neighborhood'] == 'Not assigned']

,Postcode,Borough,Neighborhood
6,M9A,Queen's Park,Not assigned


### Use the replace method to change any 'Not assigned' values in Neighborhood with values from 'Borough'. 

Make sure inplace=True to apply method on same df object

In [84]:
df.Neighborhood.replace('Not assigned',df['Borough'], inplace=True)
df[df['Neighborhood'] == 'Not assigned']

,Postcode,Borough,Neighborhood


In [85]:
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
